<a href="https://colab.research.google.com/github/rockey1006/verification/blob/main/speedup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface


In [2]:
from deepface import DeepFace


Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/facenet_weights.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/root/.deepface/weights/')

In [5]:

from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/need (1).zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/')

In [10]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/person01.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('person01/')

In [6]:
import sys
import base64
import cv2
import time
from deepface import DeepFace
from deepface.commons import functions, realtime, distance as dst
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
import numpy as np
import imageio

In [56]:
import sys
import base64
import cv2
import time
from deepface import DeepFace
from deepface.commons import functions, realtime, distance as dst
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
import numpy as np
import imageio


faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
facenet_model = Facenet.loadModel()

def load_img2(img, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
	images = np.zeros((1, image_size, image_size, 3))
	# scale pixel values
	face_pixels = img.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	images[:,:,:,:] = face_pixels
	return images

def verify_distance(img1, img2):   # assumes image is in 160x160 format
	img1 = load_img2(img1, False, False, 160)
	img2 = load_img2(img2, False, False, 160)
	img1_representation = facenet_model.predict(img1)[0,:]
	img2_representation = facenet_model.predict(img2)[0,:]
	#distance1 = dst.findCosineDistance(img1_representation, img2_representation)
	#distance2 = dst.findEuclideanDistance(img1_representation, img2_representation)
	distance3 = dst.findEuclideanDistance(dst.l2_normalize(img1_representation), dst.l2_normalize(img2_representation))
	#result = str(distance1) + ' ' + str(distance2) + ' ' + str(distance3)
	return distance3
from skimage import io

def facedetect_image(img_file):  # detects face and returns 160x160 image around face
  imagePath = img_file
  image = io.imread(imagePath)
  margin = 3  # 3 pixel extra around detected face
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(
		gray,
		scaleFactor=1.3,
		minNeighbors=3,
		minSize=(30, 30)
	)
 
  for (x, y, w, h) in faces:
		#cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    roi_color = image[y-margin:y + h+margin, x-margin:x + w+margin]
    dsize=(160,160)
    global imface
    imface = cv2.resize(roi_color, dsize, interpolation = cv2.INTER_AREA)
		#print("[INFO] Object found. Saving locally.")
		#cv2.imwrite(str(w) + str(h) + '_faces.jpg', imface1)
    encoded_string = imface 
    break
  return imface



In [57]:


def main():
  import os 

  g = os.walk(r"/content/person01/person01/batch1/") 
  img1 = facedetect_image('/content/person01/person01/000031.png/')

  for path,dir_list,file_list in g:  
    for file_name in file_list:  
      print(os.path.join(path, file_name))
      img2 = facedetect_image(os.path.join(path, file_name))
      res = verify_distance(img1,img2)
      print("euclidean_l2 distance: ",res)

if __name__ == "__main__":
    sys.exit(int(main() or 0))

/content/person01/person01/batch1/a.pngb'Blurry'.png6.png
euclidean_l2 distance:  0.0
/content/person01/person01/batch1/a.pngb'Blurry'.png9.png
euclidean_l2 distance:  0.0
/content/person01/person01/batch1/a.pngb'High_Cheekbones'.png1.png
euclidean_l2 distance:  1.2665443
/content/person01/person01/batch1/a.pngb'High_Cheekbones'.png3.png
euclidean_l2 distance:  1.2827928
/content/person01/person01/batch1/a.pngb'Brown_Hair'.png3.png
euclidean_l2 distance:  1.3052164
/content/person01/person01/batch1/a.pngb'Bushy_Eyebrows'.png4.png
euclidean_l2 distance:  1.36084
/content/person01/person01/batch1/a.pngb'Blurry'.png4.png
euclidean_l2 distance:  1.36084
/content/person01/person01/batch1/a.pngb'Heavy_Makeup'.png1.png
euclidean_l2 distance:  1.2665443
/content/person01/person01/batch1/a.pngb'Heavy_Makeup'.png8.png
euclidean_l2 distance:  1.3408746
/content/person01/person01/batch1/a.pngb'Brown_Hair'.png6.png
euclidean_l2 distance:  1.3455783
/content/person01/person01/batch1/a.pngb'Heavy_Mak

SystemExit: ignored